# a stream in the field

This is a notebook to investigate a posible excess in proper motion space in the Gaia data near Sculptor. 
It turns out that this excess is likely just an extension of the Sgr stream, which covers a large, extended region of the sky (including near sculptor). This notebook contains some plots and comparisons to show this. For my analysis, this is more of a curiosity in the field near sculptor but is sufficiently distinct in proper motion space that we should not worry about contamination from Sculptor.

# Setup

In [ ]:
using Arya, CairoMakie
using LilGuys
import CairoMakie: save
import TOML

In [ ]:
red = COLORS[6];

In [ ]:
import LinearAlgebra: normalize, ×

In [ ]:
import StatsBase: median

In [ ]:
include(ENV["DWARFS_ROOT"] * "/utils/gaia_plots.jl")

In [ ]:
include(ENV["DWARFS_ROOT"] * "/utils/gaia_filters.jl")

In [ ]:
include(ENV["DWARFS_ROOT"] * "/utils/read_iso.jl")

In [ ]:
filt_params = GaiaFilterParams(read_paramfile("processed/pm_structure.toml"))

## Loading data

the `read_gaia_stars` function simply loads in the fits file, adds tangent plane coordinates, orbit coorinates, and the elliptical radius in arcmin (with these determined from the values in filt_params).

See README.md in the data folder for notes on the Gaia query I used. I include the RUWE cut in the gaia cut since I have now downloaded ~100deg^2 of the sky.

The filters we use in this notebook for the gaia observations (the all_stars file is simply every star in Gaia within four degrees of the centre of Sculptor) are

- Parallax: $\varpi < 3 \delta \varpi$, i.e. 3-$\sigma$ consistancy with zero parallax
- RUWE < 1.3, a reasonable astrometric quality cut
- `filt_qual` combines parallax and RUWE filters
- CMD: a polygon in the parameterfile, see plots below
- Proper motion: the L2 distance in proper motion space from the approximate adopted mean proper motions (-1.5, -3.5)mas/yr is less than 1 mas / year
- `filt_all` combines all of the above.

In [ ]:
sgr_stars = read_gaia_stars("data/sgr_stream_near_scl-result.fits", filt_params)

sgr_stars[!, :filt_ruwe] = ruwe_filter(sgr_stars, filt_params)
sgr_stars[!, :filt_parallax] = parallax_simple_filter(sgr_stars, 3)
sgr_stars[!, :filt_qual] = sgr_stars.filt_ruwe .& sgr_stars.filt_parallax
sgr_stars[!, :filt_qual_strict] = sgr_stars.filt_qual .& (.!sgr_stars.in_qso_candidates) .& (.!sgr_stars.in_galaxy_candidates)

sgr_stars[!, :filt_cmd] = cmd_filter(sgr_stars, filt_params)

sgr_stars[!, :filt_pm] = pm_filter(sgr_stars, filt_params)

sgr_stars[!, :filt_all] = sgr_stars.filt_qual .& sgr_stars.filt_cmd .& sgr_stars.filt_pm
sgr_stars[!, :filt_all_strict] = sgr_stars.filt_qual_strict .& sgr_stars.filt_cmd .& sgr_stars.filt_pm

sgr_stars

In [ ]:
all_stars = read_gaia_stars("data/gaia_6deg_ruwe.fits", filt_params)

all_stars[!, :filt_ruwe] = ruwe_filter(all_stars, filt_params)
all_stars[!, :filt_parallax] = parallax_simple_filter(all_stars, 3)
all_stars[!, :filt_qual] = all_stars.filt_ruwe .& all_stars.filt_parallax
all_stars[!, :filt_qual_strict] = all_stars.filt_qual .& (.!all_stars.in_qso_candidates) .& (.!all_stars.in_galaxy_candidates)

all_stars[!, :filt_cmd] = cmd_filter(all_stars, filt_params)

all_stars[!, :filt_pm] = pm_filter(all_stars, filt_params)

all_stars[!, :filt_all] = all_stars.filt_qual .& all_stars.filt_cmd .& all_stars.filt_pm
all_stars[!, :filt_all_strict] = all_stars.filt_qual_strict .& all_stars.filt_cmd .& all_stars.filt_pm

all_stars

In [ ]:
# check that the distance filter is exactly the same as parallax / error < 3
sum((abs.(all_stars.parallax_over_error) .< 3 ) .!= all_stars.filt_parallax)

In [ ]:
# sanity check the RUWE is as described
sum((abs.(all_stars.ruwe) .< 1.3 ) .!= all_stars.filt_ruwe)

In [ ]:
dpm = @. sqrt((all_stars.pmra - filt_params.pmra)^2 + (all_stars.pmdec - filt_params.pmdec)^2);

In [ ]:
# sanity check the pm filter
sum((dpm .< 1) .!= all_stars.filt_pm)

# Plots & Analysis

## Tangent plane plots

In [ ]:
function plot_tangent_all(df; levels=10, kwargs...)
    
    fig = Figure()

    ax = xieta_axis(fig[1,1]; kwargs...)
    r_max = round(maximum(df.xi .⊕ df.eta))
    ax.limits = (-r_max, r_max, -r_max, r_max)
    
    
 
    scatter!(df.xi, df.eta, color=:black, markersize=3, alpha=0.3)
    fig
end

The following plots show the selected stars on the sky. 
The Sgr field is offset (and the density is not a good rendition...).
Note that the spatial gradient of selected stars in the Scl dataset points 
to the stream field.

In [ ]:
plot_tangent_all(sgr_stars[sgr_stars.filt_all, :], title="All filters (Sgr stream field)")

In [ ]:
plot_tangent_all(all_stars[all_stars.filt_all, :], title="All filters (Scl field)")

In [ ]:
plot_tangent_all(all_stars[all_stars.filt_qual, :], title="Parallax filters (Scl field)")

Just for reference, the above plot is all distant stars in the Scl field. 

# Filter validation

### Quality cuts

In [ ]:
import LilGuys.Plots as LP

The figure below simply plots the parallax and error for all the stars and stars satisfying the parallax cut. As expected, the parallax cut selects a wedge in this space.

In [ ]:
let
    fig = Figure()
    ax = Axis(fig[1, 1], 
        xlabel = "parallax",
        ylabel = "parallax error",
        limits = (-10, 10, 0, nothing),
        )

    scatter!(all_stars.parallax[all_stars.filt_pm], all_stars.parallax_error[all_stars.filt_pm], markersize=2, alpha=0.2, color=:black, label = "PM selected stars" => (; markersize=10))

    scatter!(all_stars.parallax[all_stars.filt_parallax .& all_stars.filt_pm], all_stars.parallax_error[all_stars.filt_parallax .& all_stars.filt_pm], markersize=2, label = "+parallax" => (; markersize=10))

    xs = 1.5 * [-5, 0, 5]
    lines!(xs, 1/3*abs.(xs), label = "3 sigma consist. with zero", color=COLORS[2])

    LP.hide_grid!(ax)

    Legend(fig[1, 2], ax)

    fig
end

In [ ]:
let
    fig = Figure()
    ax = Axis(fig[1, 1], 
        xlabel = "log RUWE", 
        ylabel = "counts",
        limits = (nothing, nothing, 1, 1e5),
        yscale=log10
        )

    bins = -0.1:0.01:0.2

    hist!(log10.(all_stars.ruwe)[all_stars.filt_ruwe], bins=bins, label="RUWE selection")
    stephist!(log10.(all_stars.ruwe), bins=bins, color=:black, label="all stars")

    vlines!(log10(1.3), color=COLORS[3], linestyle=:dash, label="threshold")

    axislegend()

    LP.hide_grid!(ax)
    fig
end

Above is a histogram of the RUWE errors. As expected, (since this is in my gaia query), no stars have a RUWE error greater than the cutoff.

## CMD

In [ ]:
# get the polygon of the CMD
cmd_x = [filt_params.cmd_cut[1:2:end]; filt_params.cmd_cut[1]]
cmd_y = [filt_params.cmd_cut[2:2:end]; filt_params.cmd_cut[2]];

In [ ]:
let
	fig = Figure()

	ax = cmd_axis(fig[1, 1])

	filt =  all_stars.filt_qual


	scatter!(all_stars.bp_rp[filt], all_stars.G[filt], markersize=2, alpha=0.3, color=:black, 
        label="+distance" => (; markersize=10))

	axislegend(position=:lt, markersize=10)

	fig
end

A scatter plot of all of the stars in the Scl field satisfying the parallax cut. 
There is a large amount of background, and the most visible CMD is from Scl.

In [ ]:
let
	fig = Figure()

	ax = cmd_axis(fig[1, 1])

	filt =  all_stars.filt_qual .& all_stars.filt_pm


	scatter!(all_stars.bp_rp[filt], all_stars.G[filt], markersize=2, alpha=1, color=:black, 
        label="distance + pm filt (Scl field)" => (; markersize=10))
    
 #    filt .&= all_stars.filt_cmd
	# scatter!(all_stars.bp_rp[filt], all_stars.G[filt], markersize=2, label=" + CMD cuts" => (; markersize=10))
	# #lines!(iso.bp_rp, iso.G .+ dm)
	# poly!(cmd_x, cmd_y, color=:transparent, strokecolor=COLORS[2], strokewidth=2)

	axislegend(position=:lt, markersize=10)

	fig
end

By adding the proper motion cut, a CMD pokes out of the noise!

In [ ]:
let
	fig = Figure()

	ax = cmd_axis(fig[1, 1])

	filt =  sgr_stars.filt_qual .& sgr_stars.filt_pm


	scatter!(sgr_stars.bp_rp[filt], sgr_stars.G[filt], markersize=2, alpha=1, color=:black, 
        label="distance + pm filt (sgr stream field)" => (; markersize=10))
    
 #    filt .&= all_stars.filt_cmd
	# scatter!(all_stars.bp_rp[filt], all_stars.G[filt], markersize=2, label=" + CMD cuts" => (; markersize=10))
	# #lines!(iso.bp_rp, iso.G .+ dm)
	# poly!(cmd_x, cmd_y, color=:transparent, strokecolor=COLORS[2], strokewidth=2)

	axislegend(position=:lt, markersize=10)

	fig
end

In the Sgr stream field, the CMD looks the same and is even more densly populated

### Isochrone & CMD cut

In [ ]:
m_h = "m2.00"
vvcrit = "0.0"
afe = "p0.0"


isos = ISOCMD("../../MIST/MIST_v1.2_vvcrit$(vvcrit)_UBVRIplus/MIST_v1.2_feh_$(m_h)_afe_$(afe)_vvcrit$(vvcrit)_UBVRIplus.iso.cmd")

In [ ]:
log_age = 10
dm = 17.1
A_BPRP = 0.1

iso = isos[log_age]
iso[!, :bp_rp] = iso.Gaia_BP_EDR3 .- iso.Gaia_RP_EDR3
iso[!, :G] = iso.Gaia_G_EDR3

iso = iso[iso.phase .< 4, :];

In [ ]:
10 ^ log_age / 1e9 # age in Gyr

In [ ]:
10 ^ (1/5 * dm + 1 - 3) # distance in kpc

In [ ]:
"log_age=$log_age, DM=$dm, [M/H]=$m_h"

In [ ]:
let
	fig = Figure()

	ax = cmd_axis(fig[1, 1])
    ax.limits = (-0.2, 2, 12, 21)

	filt =  copy(all_stars.filt_pm .& all_stars.filt_qual)


	scatter!(all_stars.bp_rp[filt], all_stars.G[filt], markersize=2, alpha=1, color=:black, 
        label="pm + dist filt" => (; markersize=10))
    
    
    filt .&= all_stars.filt_cmd
	scatter!(all_stars.bp_rp[filt], all_stars.G[filt], markersize=2, label=" + CMD cuts" => (; markersize=10))
	lines!(iso.bp_rp .+ A_BPRP, iso.G .+ dm,
        label="isochrone"
    )
	poly!(cmd_x, cmd_y, color=:transparent, strokecolor=COLORS[2], strokewidth=1, alpha=0.3, )
    
	axislegend(position=:lt)

	fig
end

The isochrone above fits the CMD well. This distance is consistant with the Stream as well. 

## Stream coordinates

In [ ]:
# Transformation described in Majewski 2003, ApJ, 599, 1082
ϕ = deg2rad(183.8)
θ = deg2rad(76.5)
ψ = deg2rad(194.1)


C = LilGuys.Rx_mat(-θ)
B = LilGuys.Rz_mat(-ψ)
D = LilGuys.Rz_mat(-ϕ)

A = B*C*D

l_sgr = 5.56
b_sgr = -14.16
xyz_sgr = LilGuys.unit_vector(l_sgr,b_sgr)

lambda_sgr, B_sgr, _ = LilGuys.cartesian_to_sky(A * xyz_sgr)

In [ ]:

l, b = all_stars.l, all_stars.b
xyz = LilGuys.unit_vector(l, b)

l, b, _ = LilGuys.cartesian_to_sky(xyz .* [1, 1, 1])

xyz = LilGuys.unit_vector(l, b)
lambda, B, _ = LilGuys.cartesian_to_sky(A * xyz);
scatter(360 .- lambda[all_stars.filt_all], B[all_stars.filt_all])

In [ ]:
# these transformations are given in the
# Belokurov et al. (2014), MNRAS, 437, 116 paper.
# The Vasiliev paper (et al. 2021, a tango for three...) uses 
# the same coordinates except for a reversed B coordinate.

function from_vasiliev_stream_coords(Λ, B)
    B *= -1
    Λ = deg2rad.(Λ)
    B = deg2rad.(B)
    α = @. atand(-0.848_462_91* cos(Λ) * cos(B) - 0.319_106_58  *sin(Λ) *cos(B) - 0.422_234_15 * sin(B),
             0.212_155_55 * cos(Λ) * cos(B) - 0.935_953_54 * sin(Λ)* cos(B) + 0.281_035_59 * sin(B))
    δ = @. asind(-0.484_871_86 * cos(Λ)* cos(B) + 0.148_868_95 * sin(Λ) * cos(B) + 0.861_822_09 * sin(B))

    return α, δ
end

function to_vasiliev_stream_coords(α, δ)
    α = deg2rad.(α)
    δ = deg2rad.(δ)
    
    Λ =@. atand(-0.935_953_54 *  cos(α) * cos(δ) − 0.319_106_58 * sin(α) * cos(δ) + 0.148_868_95 *  sin(δ),
    0.212_155_55 * cos(α) * cos(δ) − 0.848_462_91*sin(α)*cos(δ) - 0.484_871_86 *sin(δ))
    
    B = @. asind(0.281_035_59*cos(α)*cos(δ) - 0.422_234_15 * sin(α) * cos(δ) + 0.861_822_09 * sin(δ))
    B *= -1

    return Λ, B
end

In [ ]:
# sanity check inverse transform
to_vasiliev_stream_coords(from_vasiliev_stream_coords(23, 41)...)

In [ ]:
from_vasiliev_stream_coords(to_vasiliev_stream_coords(12, -25)...)

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], xlabel=L"\Lambda", ylabel=L"B")

ra, dec = all_stars.ra, all_stars.dec

Λ, B = to_vasiliev_stream_coords(ra, dec)
scatter!(Λ[all_stars.filt_all], B[all_stars.filt_all], markersize=3, label="stream field")


ra, dec = sgr_stars.ra, sgr_stars.dec
Λ, B = to_vasiliev_stream_coords(ra, dec)
scatter!(Λ[sgr_stars.filt_all], B[sgr_stars.filt_all], markersize=3, label="Scl field")


axislegend()
fig

In [ ]:
from_vasiliev_stream_coords(-75, 0) # selection of nearby stream  centre

## Radial velocities

In [ ]:
filt_rv = all_stars.filt_qual .& all_stars.filt_pm
println("rv meas:", sum(isfinite.(all_stars[filt_rv, :radial_velocity])))

filt_rv .&= isfinite.(all_stars.radial_velocity);

In [ ]:
df = all_stars[filt_rv, :]
fig = Figure()
ax = Axis(fig[1, 1], xlabel="G (mag)", ylabel = "radial velocity")

errscatter!(df.G, df.radial_velocity, yerr=df.radial_velocity_error)

fig

## Proper motions

The below plot is mainly a validation of the PM selection

In [ ]:
let
	fig = Figure()
    ax = pm_axis(fig[1, 1], dpm=11)
    ax.title = "Gaia stars within 6 degrees of Scl"
    
	scatter!(all_stars.pmra, all_stars.pmdec, markersize=1, alpha=0.2, color=:black, )
	scatter!([NaN], [NaN], markersize=5, alpha=0.2, color=:black, label = "all stars in Gaia")
    
	filt = all_stars.filt_pm 

	scatter!(all_stars.pmra[filt], all_stars.pmdec[filt], markersize=1, color=COLORS[1])
	scatter!([NaN], [NaN], markersize=5, color=COLORS[1], label = "PM selected stars")

    arc!(Point2f(filt_params.pmra, filt_params.pmdec), filt_params.dpm, -π, π, color=COLORS[2], label = "PM cut")

    leg = Legend(fig[1, 2], ax, markersize=15)
    
	fig
end

In [ ]:
let
	fig = Figure()
    ax = pm_axis(fig[1, 1], dpm=11)
    ax.title = "Scl field (Sgr CMD + PM selected)"

    filt = all_stars.filt_parallax .& all_stars.filt_cmd
	scatter!(all_stars.pmra[filt], all_stars.pmdec[filt], markersize=1, alpha=0.2, color=:black, )
	scatter!([NaN], [NaN], markersize=5, alpha=0.2, color=:black, label = "parallax filt")
    
    arc!(Point2f(filt_params.pmra, filt_params.pmdec), 1, -π, π, color=COLORS[2], label = "PM cut")
    
	fig
end

In [ ]:
let
	fig = Figure()
    ax = pm_axis(fig[1, 1], dpm=11)
    ax.title = "Sgr stream field (CMD + PM selected)"

    filt = sgr_stars.filt_parallax 
	scatter!(sgr_stars.pmra[filt], sgr_stars.pmdec[filt], markersize=1, alpha=0.2, color=:black, )
	scatter!([NaN], [NaN], markersize=5, alpha=0.2, color=:black, label = "parallax filt")
   
    arc!(Point2f(filt_params.pmra, filt_params.pmdec), 1, -π, π, color=COLORS[2], label = "PM cut")
    
	fig
end

Comparing the PM densities of both fields (when selecting by parallax & CMD) reveals that the excess of stars I noticed is much more apparent in the Sgr stream field. The proper motions are futhermore ~ consistent with the model in Vasiliev et al. (2021). The orange circle in both plots is the PM selection for this notebook.